# TP3 - Frequency Domain
ATRIM - Option Datasim

Ecole Centrale Nantes

Diana Mateus

Participants: Yassine Jamoud, Samy Haffoudhi

## GOAL

In this lab we will:
- put into practice the computation of the DFT of an image, and learn to interpret the result. 
- Design and apply bandpass and Notch filters. 
- Replicate the use of the DCT for image compression in Jpeg.

## 0. Preparation 

### 0.1 Import the required modules


In [ ]:
from scipy import ndimage
from scipy import misc
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage.io as io
from skimage.transform import resize

### 0.2 Define the main image folder
Make sure the subsequent parts of this notebook refer to this definition IMDIR. **When evaluating your notebook I should only need to change the path here** to run the entire notebook.

In [ ]:
IMDIR = "./images/"
print(os.listdir(IMDIR))

## 1. The Discrete Fourier Transform



**In the report:** Resume the steps and explain the DFTs for each image

### 1.1 Computing and visualizing the DFT  


In this exercise we will compute and visualize a DFT using numpy's fft functions:
- ```np.fft.fft2```: computes the two-dimensional FFT of an image.
- ```np.fft.fftshift```: center the frecuencies in the middle of the image
- ```np.fft.ifftshift```: reverses the centering
- ```np.fft.ifft2```: inverse fourier transform
- ```np.real```: takes the real components of a complex number
- ```np.abs```: recovers the magnitude of a complex number
- ```np.angle```: recovers the phase of a complex number 
- ```np.log```: computes the logarithm

Apply the functions above to the images in the ```Fourier``` folder. Display in a single row for each image:

**a)** the original image 

**b)** the magnitude of its fft (scaled logarithmically)

**c)** the magnitude of its **centered(shifted)** fft (scaled logarithmically)

**c)** the inverse fft of the above transformed image (all the steps shouls be reversed)

**d) For each image explain how the information in the image relates to its representation in the frequency domain**




In [ ]:
SUBDIR = os.path.join(IMDIR, "Fourier")


for root, dirnames, filenames in os.walk(SUBDIR):
    # print path to all filenames.
    for filename in filenames:
        f = os.path.join(root, filename)
        if f.endswith(('.png', '.jpg', '.jpeg','.JPG', '.tif', '.gif')):
            im = io.imread(f,as_gray=True)
            #im = resize(im,(100,100),mode='constant') # RESIZE IF TOO SLOW 
    
            im_fft = np.fft.fft2(im)
            im_fft_centered = np.fft.fftshift(im_fft)
            
            im_ftt_not_centered = np.fft.ifftshift(im_fft_centered)
            im_inverse_fft = np.fft.ifft2(im_ftt_not_centered)
            image_recovered = np.real(im_inverse_fft)
            
            fig=plt.figure(figsize=(16, 18))

            plt.subplot(141)
            plt.title('Original image')
            plt.imshow(im)
            plt.gray()
            plt.axis('off')
            
            plt.subplot(142)
            plt.title('log FFT ')
            plt.imshow(np.log(np.abs(im_fft)))
            plt.gray()
            plt.axis('off')
            
            plt.subplot(143)
            plt.title('log centered FFT ')
            plt.imshow(np.log(np.abs(im_fft_centered)))
            plt.gray()
            plt.axis('off')

            plt.subplot(144)
            plt.title('Reconstructed image')
            plt.imshow(image_recovered)
            plt.gray()
            plt.axis('off')
            
            plt.show()

Commentaires pour chaque image (dans l'ordre) :

* la strucutre de l'image (motifs carrés repetées) se refléte dans sa transformées de Fourier
* les rayures sont refletées dans la transformée de Fourier avec par exemple les pics selon les axes horizontaux et verticaux correspondant aux rayures horizontales et verticales. Ces pics sont à plus ou moins haute fréquence selon l'espacement entre les rayures
* Ici la transformée comporte majoritairement une ligne verticale correspondant au pattern régulier des pavés dans l'image
* La transformée comporte 2 lignes verticales et horizontales et principales correspondant à la strucutre régulière de l'image, le reste correspond principalement au cercle visible en haut à droite de l'image
* Encore une fois la strucutre régulière du fond se manifeste dans la transformée de Fourier, on peut aussi voir des diagonales correspondant aux rayures du zebre

## 2. Filtering in the frequency domain
**a)** Implement two functions that create a low pass and a high-pass filter frequency space for a given cutoff frequency $D_0$. The functions receive as input parameter the type of filter: 'ideal' or 'gaussian', and the filter parameters.

**b)** Show the filter frequency responses for different values of the cuttoff frequency both for the gaussian and the ideal filters. 

**c)** Filter **ONE** of the images in the ```freqfilt``` folder with the implemented functions. 

**d)** For the image in c) Show the results of the low and high pass filters  both in the frequency and the spatial domain.

**e)** Show and compare the results from the ideal and the Gaussian filter. 
**Question**: Are there any observable differences? (write answers in the ANSWERS block below)

**f)** **Question**: What happens with the DC component after a highpass filter has been applied?

**g)** Design and apply the right filter to do the following tasks:
- remove the noise of ```tower.jpg```
- smooth out the freckles of trying to keep the face sharp ```face1.jpg```
- remove out the moire pattern from  ```car-moire-pattern.tif```
- highlight the wrinkles of ```face2.jpg```
- remove the interference pattern of ```astronaut-interference.tif```
- remove the shadow from the ```text-spotshade.tif```

**Explain** the filter choice, show the spatial and frequency response and comment your findings.


**IN THE REPORT** Answer questions e), f) and g) 


In [ ]:
def lowpass_filter(D, shape, filter_type='ideal'):
    Q, P = shape
    if filter_type == 'ideal':
        lowpass = np.array([[int((u - P//2)**2 + (v - Q//2)**2 < D**2) for u in range(P)] for v in range(Q)])
    else:
        lowpass = np.array([[ np.exp(-((u - P//2)**2 + (v - Q//2)**2 )/(2*D**2)) for u in range(P)]for v in range(Q)])
    return lowpass

def highpass_filter(D,shape,filter_type='ideal'):
    Q, P = shape
    if filter_type == 'ideal':
        lowpass = np.array([[int((u - P//2)**2 + (v - Q//2)**2 > D**2) for u in range(P)] for v in range(Q)])
    else:
        lowpass = 1-lowpass_filter(D,shape,'gaussian')
    return lowpass

In [ ]:
fig=plt.figure(figsize=(5, 9))

D_values = [3, 10, 15]

counter = 1

for D in D_values:
    f_ideal = lowpass_filter(D, (40, 40))
    f_gauss = lowpass_filter(D, (40, 40), 'gauss')
    
    plt.subplot(3,2,counter)
    plt.imshow(f_ideal, cmap='gray')
    plt.axis('off')
    plt.title(f'Ideal D = {D}')
    
    plt.subplot(3,2,counter+1)
    plt.imshow(f_gauss, cmap='gray')
    plt.axis('off')
    plt.title(f'Gauss D = {D}')
    
    counter += 2
    
plt.show()

In [ ]:
SUBDIR = os.path.join(IMDIR, "Frequency")
f = os.path.join(SUBDIR, "face2.jpg")

im = io.imread(f, as_gray=True)
im = im.astype(float)

N,M = im.shape
pad_im = np.zeros((2*N,2*M))
pad_im[0:N,0:M] = im

fft = np.fft.fft2(pad_im)

fft_center = np.fft.fftshift(fft)

D=20
high = highpass_filter(D,pad_im.shape);
low = lowpass_filter(D,pad_im.shape);
high_gauss = highpass_filter(D,pad_im.shape, 'gaussian')
low_gauss = lowpass_filter(D,pad_im.shape,'gaussian');

fft_low = low * fft_center;
fft_high = high * fft_center;
fft_low_gauss = low_gauss * fft_center;
fft_high_gauss = high_gauss * fft_center

inverse_fft_low = np.fft.ifftshift(fft_low)
inverse_fft_high = np.fft.ifftshift(fft_high)
inverse_fft_low_gauss = np.fft.ifftshift(fft_low_gauss)
inverse_fft_high_gauss = np.fft.ifftshift(fft_high_gauss)
    
inverse_fft_low = np.fft.ifft2(inverse_fft_low)
inverse_fft_high = np.fft.ifft2(inverse_fft_high)
inverse_fft_low_gauss = np.fft.ifft2(inverse_fft_low_gauss)
inverse_fft_high_gauss = np.fft.ifft2(inverse_fft_high_gauss)

im_low = np.real(inverse_fft_low)
im_high = np.real(inverse_fft_high)
im_low_gauss = np.real(inverse_fft_low_gauss)
im_high_gauss = np.real(inverse_fft_high_gauss)

final_low = im_low[:N,:M]
final_high = im_high[:N,:M]
final_low_gauss = im_low_gauss[:N,:M]
final_high_gauss = im_high_gauss[:N,:M]

fig=plt.figure(figsize=(16, 8))

plt.subplot(2,5,1)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(2,5,2)
plt.imshow(final_low, cmap='gray')
plt.axis('off')
plt.title('Low Pass Ideal')

plt.subplot(2,5,3)
plt.imshow(final_high, cmap='gray')
plt.axis('off')
plt.title('High Pass Ideal')

plt.subplot(2,5,4)
plt.imshow(final_low_gauss, cmap='gray')
plt.axis('off')
plt.title('Low Pass Gauss')

plt.subplot(2,5,5)
plt.imshow(final_high_gauss, cmap='gray')
plt.axis('off')
plt.title('High Pass Gauss')

plt.subplot(2,5,6)
plt.imshow(np.log(np.abs(fft_center)), cmap='gray')
plt.axis('off')
plt.title('Original')

plt.subplot(2,5,7)
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(final_low)))), cmap='gray')
plt.axis('off')
plt.title('Low Pass Ideal')

plt.subplot(2,5,8)
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(final_high)))), cmap='gray')
plt.axis('off')
plt.title('High Pass Ideal')

plt.subplot(2,5,9)
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(final_low_gauss)))), cmap='gray')
plt.axis('off')
plt.title('Low Pass Gauss')

plt.subplot(2,5,10)
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(final_high_gauss)))), cmap='gray')
plt.axis('off')
plt.title('High Pass Gauss')

plt.show()

* Nous observons alors à l'aide des différentes images ci-dessus que les filtres Gaussiens permettent bien d'éliminer les artefacts sur l'image, par exemple les lignes verticales sur le côté gauche du visage. Ils sont dus aux variation brusques de la fonction de transfert, le filtre gaussien permet de remédier à ce problème.
* Après application d'un filtre passe haut la composante continue est supprimée

In [ ]:
def apply_filter(im, f):
    N, M = im.shape
    im_padded = np.zeros((2*N, 2*M))
    im_padded[0:N,0:M] = im
    im_fft = np.fft.fft2(im_padded)
    im_fft = np.fft.fftshift(im_fft)
    res_fft = f * im_fft
    res = np.fft.ifft2(np.fft.ifftshift(res_fft))
    res = np.real(res)
    res = res[:N, :M]
    
    return res

In [ ]:
SUBDIR = os.path.join(IMDIR, "Frequency")
fig = plt.figure(figsize=(30, 36))

# tower

f = os.path.join(SUBDIR, "tower.jpg")
im = io.imread(f, as_gray=True)
im = im.astype(float)

H, W = im.shape
f_denoise = np.ones((2*H, 2*W)) # à remplacer par le filtre adapté
res = apply_filter(im, f_denoise)

plt.subplot(6,4,1)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(6,4,2)
plt.imshow(res, cmap = 'gray')
plt.axis('off')
plt.title('Filtered Spatial')

plt.subplot(6,4,3)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im)))), cmap='gray')
plt.axis('off')
plt.title('Original Frequency')

plt.subplot(6,4,4)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(res)))), cmap='gray')
plt.axis('off')
plt.title('Filtered Frequency')

# face 1

f = os.path.join(SUBDIR, "face1.jpg")
im = io.imread(f, as_gray=True)
im = im.astype(float)

H, W = im.shape
f_denoise = lowpass_filter(80, (2*H, 2*W), 'gauss')
res = apply_filter(im, f_denoise)

plt.subplot(6,4,5)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(6,4,6)
plt.imshow(res, cmap = 'gray')
plt.axis('off')
plt.title('Filtered Spatial')

plt.subplot(6,4,7)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im)))), cmap='gray')
plt.axis('off')
plt.title('Original Frequency')

plt.subplot(6,4,8)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(res)))), cmap='gray')
plt.axis('off')
plt.title('Filtered Frequency')

# face 2

f = os.path.join(SUBDIR, "face2.jpg")
im = io.imread(f, as_gray=True)
im = im.astype(float)

H, W = im.shape
f_denoise = lowpass_filter(60, (2*H, 2*W))
res = apply_filter(im, f_denoise)

plt.subplot(6,4,9)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(6,4,10)
plt.imshow(res, cmap = 'gray')
plt.axis('off')
plt.title('Filtered Spatial')

plt.subplot(6,4,11)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im)))), cmap='gray')
plt.axis('off')
plt.title('Original Frequency')

plt.subplot(6,4,12)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(res)))), cmap='gray')
plt.axis('off')
plt.title('Filtered Frequency')

# Astronaut

f = os.path.join(SUBDIR, "astronaut-interference.tif")
im = io.imread(f, as_gray=True)
im = im.astype(float)

H, W = im.shape
f_denoise = np.ones((2 * H, 2 * W))
res = apply_filter(im, f_denoise)

plt.subplot(6,4,13)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(6,4,14)
plt.imshow(res, cmap = 'gray')
plt.axis('off')
plt.title('Filtered Spatial')

plt.subplot(6,4,15)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im)))), cmap='gray')
plt.axis('off')
plt.title('Original Frequency')

plt.subplot(6,4,16)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(res)))), cmap='gray')
plt.axis('off')
plt.title('Filtered Frequency')

# Text

f = os.path.join(SUBDIR, "text-spotshade.tif")
im = io.imread(f, as_gray=True)
im = im.astype(float)

H, W = im.shape
f_denoise = np.ones((2 * H, 2 * W))
res = apply_filter(im, f_denoise)

plt.subplot(6,4,17)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(6,4,18)
plt.imshow(res, cmap = 'gray')
plt.axis('off')
plt.title('Filtered Spatial')

plt.subplot(6,4,19)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im)))), cmap='gray')
plt.axis('off')
plt.title('Original Frequency')

plt.subplot(6,4,20)
plt.imshow(res, cmap = 'gray')
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(res)))), cmap='gray')
plt.axis('off')
plt.title('Filtered Frequency')

plt.show()

Pour chaque image dont nous souhaitons enlever le bruit, nous recherchons sur la transformée de Fourier les composantes responsables du bruit et nous concevons un filtre ayant pour effet d'annuler ces composantes. Nous ne sommes cependant pas parvenu à obtenir des résultats satisfaisant en pratique et nous avons alors juste conservé la structure dans code dans la cell ci-dessus.

##  3. Towards JPEG Compression

**Introduction**

One of the crucial steps of JPEG Compression is the quantization in frequency space. The procedure follows the next steps:

**i)** An image is first subdivided in 8x8 subblocks (assuming the original image has sides whose length is a multiple of 8, if it is not the case just resize it).

**ii)** A frequency transformation is computed for each block (computed with a discrete cosine transform DCT II )
```
fftpack.dct(fftpack.dct(im.T, norm='ortho').T, norm='ortho')
```
**iii)** The DCT results are then divided by the elements of a predefined quantization matrix, then rounded and stored as integers. **Hint:** use functions  ```round()```, ```astype(np.int32)```

**iv)** The result of quantization (which should have many zeros) is then compressed Run Lenght Coding and Huffman coding to reduce the size. The compressed values are saved together with the quantization table.

**v)** The reconstruction of the image implies inverting the encoding, the quantization and the dct.

See more info in this video.
https://www.youtube.com/watch?v=Q2aEzeMDHMA



**What to implement**

In this part you will reproduce the steps regarding the DCT from the JPEG compression procedure, that is steps i) to iii). Pick a grayscale image of your choice, resize it if necessary.

**a)** Compute the frequency quantization step, using the provided ```quantization_table``` and ``dct_2d`` functions.

**b)** Restore the image from its quantized frequency representation.

**c)** Show the frequency spectrum  before and after the quantization.

**d)** Show:
- the original image
- the quantized image 
- the difference between the two.

**e)** Repeat the above steps but resizing the input image to 128,128 first. What do you observe ?

**f)** Replace the given DCT function by your own basis implementation. Explain any difference.


**IN THE REPORT** Comment on your findings for questions d), e) and f)



In [ ]:
from skimage.transform import resize
import scipy
from scipy import fftpack


def dct_2d(im):
    return fftpack.dct(fftpack.dct(im.T, norm='ortho').T, norm='ortho')

def idct_2d(im):
    return fftpack.idct(fftpack.idct(im.T, norm='ortho').T, norm='ortho')

def quantization_table(component='lum'):
    # Quantization Table for: Photoshop 
    # (http://www.impulseadventure.com/photo/jpeg-quantization.html)
    if component == 'lum':
        q = np.array([[2, 2, 2, 2, 3, 4, 5, 6],
                      [2, 2, 2, 2, 3, 4, 5, 6],
                      [2, 2, 2, 2, 4, 5, 7, 9],
                      [2, 2, 2, 4, 5, 7, 9, 12],
                      [3, 3, 4, 5, 8, 10, 12, 12],
                      [4, 4, 5, 7, 10, 12, 12, 12],
                      [5, 5, 7, 9, 12, 12, 12, 12],
                      [6, 6, 9, 12, 12, 12, 12, 12]])
    elif component == 'chrom':
        q = np.array([[3, 3, 5, 9, 13, 15, 15, 15],
                      [3, 4, 6, 11, 14, 12, 12, 12],
                      [5, 6, 9, 14, 12, 12, 12, 12],
                      [9, 11, 14, 12, 12, 12, 12, 12],
                      [13, 14, 12, 12, 12, 12, 12, 12],
                      [15, 12, 12, 12, 12, 12, 12, 12],
                      [15, 12, 12, 12, 12, 12, 12, 12],
                      [15, 12, 12, 12, 12, 12, 12, 12]])
    else:
        raise ValueError((
            "component should be either 'lum' or 'chrom', "
            "but '{comp}' was found").format(comp=component))

    return q


im = scipy.misc.ascent().astype(float)
H, W = im.shape

In [ ]:
table = quantization_table()
quantized = np.zeros((H, W)).astype('int32')

# Quantization
for i in range(H//8):
    for j in range(W//8):
        block = im[8*i:8*(i+1), 8*j:8*(j+1)]
        block_dct = dct_2d(block)
        quantized[8*i:8*(i+1), 8*j:8*(j+1)] = np.round(block_dct / table)
        
# en pratique pour la phase de compression, on applique maintenant par exemple le Huffman coding

# Restoration
im_restored = np.zeros((H, W))
for i in range(H//8):
    for j in range(W//8):
        im_restored[8*i:8*(i+1), 8*j:8*(j+1)] = idct_2d(table * quantized[8*i:8*(i+1), 8*j:8*(j+1)])

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.subplot(2,3,1)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(2,3,2)
plt.imshow(im_restored, cmap = 'gray')
plt.axis('off')
plt.title('Restored')

plt.subplot(2,3,3)
plt.imshow(abs(im - im_restored), cmap = 'gray')
plt.axis('off')
plt.title('Difference')

plt.subplot(2,3,4)
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im)))), cmap='gray')
plt.axis('off')
plt.title('Original fft')

plt.subplot(2,3,5)
plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_restored)))), cmap = 'gray')
plt.axis('off')
plt.title('Restored fft')

plt.show()

In [ ]:
im = resize(im,(128,128),mode='constant')
H, W = im.shape

quantized = np.zeros((H, W)).astype('int32')

# Quantization
for i in range(H//8):
    for j in range(W//8):
        block = im[8*i:8*(i+1), 8*j:8*(j+1)]
        block_dct = dct_2d(block)
        quantized[8*i:8*(i+1), 8*j:8*(j+1)] = np.round(block_dct / table)

# Restoration
im_restored = np.zeros((H, W))
for i in range(H//8):
    for j in range(W//8):
        im_restored[8*i:8*(i+1), 8*j:8*(j+1)] = idct_2d(table * quantized[8*i:8*(i+1), 8*j:8*(j+1)])
        
fig = plt.figure(figsize=(20, 36))

plt.subplot(1,3,1)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(1,3,2)
plt.imshow(im_restored, cmap = 'gray')
plt.axis('off')
plt.title('Restored')

plt.subplot(1,3,3)
plt.imshow(abs(im - im_restored), cmap = 'gray')
plt.axis('off')
plt.title('Difference')

plt.show()

On remarque que suite au resizing le bruit semble plus marqué, que ce soit en terme de forme (ce qui était attendu, les pixels étant plus gros) mais également en terme de magnitude puisqu'on peut observer des pixels plus clairs que précedemment ce qui signifie un plus grand écart d'intensité entre les deux images.

In [ ]:
def get_T(i, j):
    if i == 0:
        return 1 / np.sqrt(8)
    return np.sqrt(1 / 4) * np.cos((2 * j + 1) * i * np.pi / 16)

def dct_matrix():
    T = np.array([[get_T(i, j) for j in range(8)] for i in range(8)])
    return T

def my_dct_2d(im):
    T = dct_matrix()
    M = im - 128
    D = T @ M @ T.T
    return D

def my_idct_2d(im):
    T = dct_matrix()
    return np.round(T.T @ im @ T) + 128

In [ ]:
im = scipy.misc.ascent().astype(float)
H, W = im.shape

quantized = np.zeros((H, W)).astype('int32')

# Quantization
for i in range(H//8):
    for j in range(W//8):
        block = im[8*i:8*(i+1), 8*j:8*(j+1)]
        block_dct = my_dct_2d(block)
        quantized[8*i:8*(i+1), 8*j:8*(j+1)] = np.round(block_dct / table)

# Restoration
im_restored = np.zeros((H, W))
for i in range(H//8):
    for j in range(W//8):
        im_restored[8*i:8*(i+1), 8*j:8*(j+1)] = my_idct_2d(table * quantized[8*i:8*(i+1), 8*j:8*(j+1)])
        
fig = plt.figure(figsize=(20, 36))

plt.subplot(1,3,1)
plt.imshow(im, cmap = 'gray')
plt.axis('off')
plt.title('Original')

plt.subplot(1,3,2)
plt.imshow(im_restored, cmap = 'gray')
plt.axis('off')
plt.title('Restored')

plt.subplot(1,3,3)
plt.imshow(abs(im - im_restored), cmap = 'gray')
plt.axis('off')
plt.title('Difference')

plt.show()

Nous n'observons pas vraiment de différence significative entre les résultats obtenus avec notre implémentation de dct et celle fournie.